We have used a few resources to help us make this algorithm,

https://arpitbhayani.me/blogs/genetic-knapsack/

https://medium.com/koderunners/genetic-algorithm-part-3-knapsack-problem-b59035ddd1d6

https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python

You will find that none of the code has been copied from anywhere, these resources were used to help us only.

-> we have also used the help of ChatGPT for when we encountered errors

In [1]:
from random import choice, randrange, random
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


FEEL FREE TO PLAY WITH THE PARAMETERS, WE FOUND THAT THIS WORKS WELL


In [2]:
n = 10 # number of items
max_val = 4 # modular param for item val
max_wgt = 10 # param for item weight
wgt_limit = 50 # set limit for the sack (weight limit)
solutions_count = 100

In [3]:
items = {} # dict of items, each has weight and value (dict of lists)

val = [i for i in range(max_wgt + 1)]
wgt = [i for i in range(max_val + 1)]

def construct_pop(n, max_val, max_wgt):
    m = [i for i in range(max_val)] # list of possible values
    p = [i for i in range(max_wgt)] # list of possible weights

    item_val = [randrange(1, len(m) + 1) for i in range(n)] # random list of val for n items
    item_wgt = [randrange(1, len(p) + 1) for i in range(n)] # random list of weights ..

    item_vw = [(val, wgt) for val, wgt in zip(item_val, item_wgt)] # make tuple (x, y) of val and weight

    for idx, i in enumerate(item_vw): # get index and populate dict 'items'
        items[idx] = i # item i = (val, wgt)

    print("List of items (item number : (value, weight))")
    print(f"Number of items: {n}")
    print(f"Maximum weight: {max_wgt}")
    print(f"Maximum value: {max_val}")
    print(items) # check if everything went as intended (spoilers it did)

    return(items)

In [4]:
pop_0 = construct_pop(n, max_val = max_val, max_wgt = max_wgt)

List of items (item number : (value, weight))
Number of items: 10
Maximum weight: 10
Maximum value: 4
{0: (1, 1), 1: (1, 9), 2: (4, 1), 3: (2, 6), 4: (1, 6), 5: (2, 1), 6: (2, 7), 7: (3, 2), 8: (4, 10), 9: (4, 4)}


In [5]:
# in case we don't want to go random, we can use a predefined list of items 

items_non_random = {0: (2, 3), 1: (3, 1), 2: (4, 5), 3: (3, 10), 4: (2, 8), 5: (4, 7), 6: (2, 7), 7: (3, 8), 8: (4, 9), 9: (2, 5)}

In [6]:
def bit_strings(sol, pop_size):
    bit_string_arr = []
    for i in range(sol):
        bit_string = [randrange(0, 2) for i in range(pop_size)] # 0 or 1, randomly, length is number of items
        bit_string_arr.append(bit_string)
    bit_string_arr = np.array(bit_string_arr)

    return bit_string_arr

solutions_initial = bit_strings(solutions_count, n)

In [7]:
def set_score(bit_strings, items, weight_limit):
    scores = []
    item_id = [item for item in items.keys()]
    item_val = [item[0] for item in items.values()]
    item_wgt = [item[1] for item in items.values()]

    for solution in bit_strings:
        total_val = 0
        total_wgt = 0
        for idx, bit in enumerate(solution):
            if bit == 1:
                total_val += item_val[idx]
                total_wgt += item_wgt[idx]
        if total_wgt > weight_limit:
            total_val = 0

        scores.append(total_val)

    return scores

scores = set_score(solutions_initial, pop_0, wgt_limit)

In [8]:
# this is essentially my elitism function.

def regressed_safeguard(item_list, prev_solutions, post_solutions):
    post_solutions_copy = post_solutions.copy()
    prev_solutions_copy = prev_solutions.copy()

    prev_scores = set_score(prev_solutions_copy, item_list, wgt_limit)
    max_score_prev = max(prev_scores)
    prev_best_sol = prev_solutions[np.argmax(prev_scores)].copy()

    post_scores = set_score(post_solutions_copy, item_list, wgt_limit)
    max_score_post = max(post_scores)

    if max_score_post < max_score_prev:
        worst_index_post = np.argmin(post_scores)
        post_solutions_copy[worst_index_post] = prev_best_sol.copy()
        
        return post_solutions_copy

    else:
        return prev_solutions

In [9]:
# https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python
# the above link is what helped us implement the roulette wheel. how it works is simple, we generate some number between 0 and 1,
# then we iterate over all probabilities, add them until their sum exceeds the random number. This is the selected number.
# the fact we have also set bad solutions (weight exceeded) to 0 means they will NEVER be selected here, which is good

def roulette_prob_wheel(pop, solution_scores):
    total_score = np.sum(solution_scores) # sum of all the scores (used for probability wheel :) )
    probabilities = solution_scores / total_score # divides each score by the total, gets probability (proportion)

    survival = []
    for i in range(len(pop)):
        rand = random()
        prob_sum = 0 # we set this equal to 0, add until we exceed "rand"
    
        for idx, prob in enumerate(probabilities):
            prob_sum += prob # add prob to sum until exceed
            if prob_sum > rand:
                survival.append(pop[idx]) # appends good item to survivor list
                break

    return survival # returns 500, == original solution size, on the right track

selection = roulette_prob_wheel(solutions_initial, scores)

In [10]:
def crossover(item_list, sol_size, pop, cross_rate): # we will set a rate for crossovers, this way they do not happen every time.
    crossover_results = []
    crossover_results_safe = None
    parents = [(pop[i], pop[i + 1]) for i in range(0, len(pop), 2)] # gets parent pairs, done "2 by 2" -> 250 len
    cross_point = randrange(1, sol_size) # for each bit string, different crossover index (never 0 or 20)
    for idx, parent in enumerate(parents):
        rand = random() # between 0 and 1n
        child_1 = parents[idx][0]
        child_2 = parents[idx][1]
        if rand < cross_rate: # crossover only happens if below defined rate (args)
            child_1 = np.hstack((parent[0][:cross_point], parent[1][cross_point:])) # np.hstack just concats 2 vectors of 1 row
            child_2 = np.hstack((parent[1][:cross_point], parent[0][cross_point:]))
            crossover_results.append(child_1)
            crossover_results.append(child_2)
        else:
            crossover_results.append(child_1)
            crossover_results.append(child_2)
            
    crossover_results_safe = regressed_safeguard(item_list, pop, crossover_results)

    # crossover_results = np.concatenate(crossover_results)
    return crossover_results_safe

cross_results = crossover(pop_0, 20, selection, 0.5)

In [25]:
def mutate(item_list, sol_size, pop, mutate_rate):
    mutated_results = []
    mutated_results_safe = None
    for bit_string in pop:
        mutated_bit_string = bit_string.copy()  # Make a copy to avoid modifying the original
        for idxx, bit in enumerate(mutated_bit_string):
            rand = random()
            if rand < mutate_rate:
                mutated_bit_string[idxx] = abs(mutated_bit_string[idxx] - 1)
        mutated_results.append(mutated_bit_string)

    mutated_results_safe = regressed_safeguard(item_list, pop, mutated_results)

    return mutated_results_safe

mut = mutate(pop_0, solutions_count, cross_results, 0.1)

mut_score = set_score(mut, pop_0, wgt_limit)

print(max(mut_score))

print(mut[0])

22
[1 0 1 0 1 0 1 1 1 0]


In [45]:
cities = [
    "Paris", "Berlin", "London", "Madrid", "Rome", "Amsterdam", "Lisbon", "Prague", "Vienna", "Stockholm",
]

def genetic_algorithm(iterations, wgt_limit, pop_size, cross_rate, mutate_rate, verbose=True, random=False):
    best_solution = None
    best_score = float('-inf')
    selected_items = None

    if random:
        items = construct_pop(n, max_val, max_wgt)
    else:
        items = {0: (2, 3), 1: (3, 1), 2: (4, 5), 3: (3, 10), 4: (2, 8), 5: (4, 7), 6: (2, 7), 7: (3, 8), 8: (4, 9),
                 9: (2, 5)}

    for i in range(iterations):
        solutions = bit_strings(pop_size, len(items))
        scores = set_score(solutions, items, wgt_limit)

        best_idx = np.argmax(scores)
        max_score = scores[best_idx]

        if max_score > best_score:
            best_solution = solutions[best_idx]
            best_score = max_score
            selected_items = [item for j, item in enumerate(items.values()) if best_solution[j] == 1]

        if verbose:
            print(f"Iteration {i}: current best solution: {best_solution}, score: {best_score}")

    if verbose:
        print(f"\nBest Solution: {best_solution}, Score: {best_score}")
        print("Selected Items:")
        for item in selected_items:
            print(f"Item {item}: Value {item[0]}, Weight {item[1]}")

    return best_solution, best_score, selected_items

best_solution, best_score, selected_items = genetic_algorithm(300, 25, 10, 0.6, 0.2, verbose=True)
print(f"\nBest Solution: {best_solution}")
weights = [item[1] for item in selected_items]
values = [item[0] for item in selected_items]
values_sum = np.sum(values)
weights_sum = np.sum(weights)
print("Selected Items:")
for item in selected_items:
    print(f"Item {item}: Value {item[0]}, Weight {item[1]}")

print(f"total value of solution : {values_sum}")
print(f"total weight of solution : {weights_sum}")

Iteration 0: current best solution: [0 1 1 0 0 0 1 0 1 0], score: 13
Iteration 1: current best solution: [0 1 1 0 0 0 1 0 1 0], score: 13
Iteration 2: current best solution: [0 1 1 0 0 0 1 0 1 0], score: 13
Iteration 3: current best solution: [0 1 1 0 0 0 1 0 1 0], score: 13
Iteration 4: current best solution: [0 1 1 0 0 0 1 0 1 0], score: 13
Iteration 5: current best solution: [0 1 1 0 0 0 1 0 1 0], score: 13
Iteration 6: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 7: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 8: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 9: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 10: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 11: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 12: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 13: current best solution: [0 1 1 0 0 1 0 1 0 0], score: 14
Iteration 14: current best sol